deberta + lstm

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_absolute_error, matthews_corrcoef, r2_score, confusion_matrix, roc_auc_score
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

DEVICE     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PRETRAINED = 'microsoft/deberta-base'
MAX_LEN    = 64
BATCH_SIZE = 4
EPOCHS     = 3
LR         = 2e-5
DATA_PATH  = r'D:\Thesis\16-7-25\thesis 2.csv'

df = pd.read_csv(DATA_PATH)
X = df['Text'].astype(str).tolist()
y = df['Label'].astype(int).tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        enc = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].squeeze(0),
            'attention_mask': enc['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
train_ds = SentimentDataset(X_train, y_train, tokenizer, MAX_LEN)
test_ds  = SentimentDataset(X_test,  y_test,  tokenizer, MAX_LEN)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)

class DebertaLSTMClassifier(nn.Module):
    def __init__(self, deberta_model, hidden_dim, num_classes):
        super().__init__()
        self.deberta = deberta_model
        H = deberta_model.config.hidden_size
        self.bilstm = nn.LSTM(H, hidden_dim, batch_first=True, bidirectional=True)
        self.drop   = nn.Dropout(0.3)
        self.fc     = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            out = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        x = out.last_hidden_state
        x, _ = self.bilstm(x)
        x = torch.mean(x, dim=1)
        x = self.drop(x)
        return self.fc(x)

deberta_model = AutoModel.from_pretrained(PRETRAINED)
deberta_model.gradient_checkpointing_enable()
model = DebertaLSTMClassifier(deberta_model.to(DEVICE), hidden_dim=64, num_classes=len(set(y))).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

train_accuracies = []

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in train_loader:
        ids = batch['input_ids'].to(DEVICE)
        mask = batch['attention_mask'].to(DEVICE)
        lbls = batch['label'].to(DEVICE)

        optimizer.zero_grad()
        logits = model(ids, mask)
        loss = criterion(logits, lbls)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == lbls).sum().item()
        total += lbls.size(0)

    acc = correct / total
    train_accuracies.append(acc)
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss/len(train_loader):.4f} - Train Acc: {acc:.4f}")

model.eval()
y_true, y_pred, y_prob = [], [], []

with torch.no_grad():
    for batch in test_loader:
        ids = batch['input_ids'].to(DEVICE)
        mask = batch['attention_mask'].to(DEVICE)
        lbls = batch['label'].to(DEVICE)

        logits = model(ids, mask)
        probs  = torch.softmax(logits, dim=1)
        preds  = torch.argmax(probs, dim=1)

        y_true.extend(lbls.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

acc  = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec  = recall_score(y_true, y_pred, average='macro')
f1   = f1_score(y_true, y_pred, average='macro')
mae  = mean_absolute_error(y_true, y_pred)
mcc  = matthews_corrcoef(y_true, y_pred)
r2   = r2_score(y_true, y_pred)

print("\n📊 Final Test Metrics:")
print(f" Accuracy       : {acc:.4f}")
print(f" Precision      : {prec:.4f}")
print(f" Recall         : {rec:.4f}")
print(f" F1 Score       : {f1:.4f}")
print(f" Mean Abs Error : {mae:.4f}")
print(f" MCC            : {mcc:.4f}")
print(f" R\u00b2 Score       : {r2:.4f}")

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

plt.figure(figsize=(6, 4))
plt.plot(range(1, EPOCHS + 1), train_accuracies, marker='o', color='green')
plt.title('Training Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

if len(set(y_true)) == 2:
    auc = roc_auc_score(y_true, [p[1] for p in y_prob])
    print(f"ROC AUC Score: {auc:.4f}")


ALBERT + BiGRU 

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PRETRAINED = 'albert-base-v2'
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-5
DATA_PATH = r'D:\Thesis\16-7-25\thesis 2.csv'  # Your CSV file path

# Load dataset
df = pd.read_csv(DATA_PATH)
texts = df['Text'].astype(str).tolist()
labels = df['Label'].astype(int).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Prepare tokenizer and data loaders
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
train_ds = SentimentDataset(X_train, y_train, tokenizer, MAX_LEN)
test_ds = SentimentDataset(X_test, y_test, tokenizer, MAX_LEN)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# Model: ALBERT + BiGRU
class AlbertGRUClassifier(nn.Module):
    def __init__(self, albert_model, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.albert = albert_model
        hidden_size = albert_model.config.hidden_size
        self.gru = nn.GRU(
            input_size=hidden_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.3 if num_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.albert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # [batch, seq_len, hidden_size]
        gru_out, _ = self.gru(sequence_output)       # [batch, seq_len, hidden_dim*2]
        avg_pool = torch.mean(gru_out, dim=1)        # mean over seq_len
        x = self.dropout(avg_pool)
        logits = self.classifier(x)
        return logits

# Instantiate model
albert_model = AutoModel.from_pretrained(PRETRAINED)
model = AlbertGRUClassifier(albert_model, hidden_dim=128, num_layers=1, num_classes=len(set(labels))).to(DEVICE)

# Optimizer and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

# Track training accuracy for plotting
train_acc_list = []

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['label'].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)

    epoch_acc = total_correct / total_samples
    train_acc_list.append(epoch_acc)
    print(f"Epoch {epoch+1}/{EPOCHS} — Loss: {total_loss/len(train_loader):.4f} — Accuracy: {epoch_acc:.4f}")

# Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['label'].to(DEVICE)

        outputs = model(input_ids, attention_mask)
        preds = outputs.argmax(dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("\nFinal Test Metrics:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Plot training accuracy curve
plt.figure(figsize=(8, 5))
plt.plot(range(1, EPOCHS + 1), train_acc_list, marker='o', label='Train Accuracy')
plt.title('Training Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(range(1, EPOCHS + 1))
plt.grid(True)
plt.legend()
plt.show()

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


ELECTRA + BiLSTM

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PRETRAINED = "google/electra-base-discriminator"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-5
DATA_PATH = r"D:\Thesis\16-7-25\thesis 2.csv"

# Dataset
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# Model
class ElectraBiLSTM(nn.Module):
    def __init__(self, electra_model, hidden_dim, num_layers, num_classes):
        super().__init__()
        self.electra = electra_model
        hidden_size = electra_model.config.hidden_size
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.3 if num_layers > 1 else 0.0,
        )
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        out = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = out.last_hidden_state
        lstm_out, _ = self.lstm(sequence_output)
        avg_pool = torch.mean(lstm_out, dim=1)
        x = self.dropout(avg_pool)
        logits = self.classifier(x)
        return logits

# Load Data
df = pd.read_csv(DATA_PATH)
texts = df["Text"].astype(str).tolist()
labels = df["Label"].astype(int).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
train_ds = SentimentDataset(X_train, y_train, tokenizer, MAX_LEN)
test_ds = SentimentDataset(X_test, y_test, tokenizer, MAX_LEN)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

electra_model = AutoModel.from_pretrained(PRETRAINED)
model = ElectraBiLSTM(electra_model, hidden_dim=128, num_layers=1, num_classes=len(set(labels))).to(DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

train_acc_list = []

# Train
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_acc = correct / total
    train_acc_list.append(epoch_acc)
    print(f"Epoch {epoch+1}/{EPOCHS} — Loss: {total_loss/len(train_loader):.4f} — Accuracy: {epoch_acc:.4f}")

# Evaluate
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        outputs = model(input_ids, attention_mask)
        preds = outputs.argmax(dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average="macro")
rec = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")

print("\nFinal Test Metrics:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

# Plot training accuracy curve
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.plot(range(1, EPOCHS+1), train_acc_list, marker='o', label='Train Accuracy')
plt.title('Training Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.xticks(range(1, EPOCHS+1))
plt.grid(True)
plt.legend()
plt.show()

# Confusion matrix
import seaborn as sns
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


BERT + SVM 

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

PRETRAINED = "bert-base-uncased"
MAX_LEN = 128
BATCH_SIZE = 32
DATA_PATH = r"D:\Thesis\16-7-25\thesis 2.csv"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data
df = pd.read_csv(DATA_PATH)
texts = df["Text"].astype(str).tolist()
labels = df["Label"].astype(int).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

# Tokenizer + model for embeddings
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
model = AutoModel.from_pretrained(PRETRAINED).to(device)
model.eval()

def embed_texts(texts, tokenizer, model, max_len):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors="pt", max_length=max_len, truncation=True, padding="max_length")
        inputs = {k: v.to(device) for k,v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_emb = outputs.last_hidden_state[:,0,:].cpu().numpy()
            embeddings.append(cls_emb.squeeze())
    return np.array(embeddings)

print("Embedding training data...")
X_train_emb = embed_texts(X_train, tokenizer, model, MAX_LEN)
print("Embedding test data...")
X_test_emb = embed_texts(X_test, tokenizer, model, MAX_LEN)

# Train SVM
svm = SVC(kernel="linear")
svm.fit(X_train_emb, y_train)

# Train accuracy
y_train_pred = svm.predict(X_train_emb)
train_acc = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_acc:.4f}")

# Predict and evaluate
y_pred = svm.predict(X_test_emb)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average="macro")
rec = recall_score(y_test, y_pred, average="macro")
f1 = f1_score(y_test, y_pred, average="macro")

print("Test Accuracy:", acc)
print("Test Precision:", prec)
print("Test Recall:", rec)
print("Test F1:", f1)

# Confusion matrix plot
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()


ERNIE + CNN

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    matthews_corrcoef, mean_absolute_error, r2_score,
    confusion_matrix, roc_auc_score
)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import label_binarize
from itertools import cycle

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PRETRAINED = "nghuyong/ernie-1.0"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3
LR = 2e-5
DATA_PATH = "thesis.csv"

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer.encode_plus(
            self.texts[idx],
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

class ERNIE_CNN(nn.Module):
    def __init__(self, ernie_model, num_classes):
        super().__init__()
        self.ernie = ernie_model
        hidden_size = ernie_model.config.hidden_size

        self.conv1 = nn.Conv1d(in_channels=hidden_size, out_channels=100, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(100, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.ernie(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state

        x = hidden_states.permute(0, 2, 1)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x).squeeze(-1)
        x = self.dropout(x)
        logits = self.classifier(x)
        return logits

df = pd.read_csv(DATA_PATH)
texts = df["Text"].astype(str).tolist()
labels = df["Label"].astype(int).tolist()
num_classes = len(set(labels))

X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
train_ds = SentimentDataset(X_train, y_train, tokenizer, MAX_LEN)
test_ds = SentimentDataset(X_test, y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

ernie_model = AutoModel.from_pretrained(PRETRAINED)
model = ERNIE_CNN(ernie_model, num_classes=num_classes).to(DEVICE)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

train_true_all = []
train_pred_all = []

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        train_true_all.extend(labels.cpu().numpy())
        train_pred_all.extend(preds.cpu().numpy())

    train_acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} — Loss: {total_loss/len(train_loader):.4f} — Train Accuracy: {train_acc:.4f}")

# Calculate train metrics
train_prec = precision_score(train_true_all, train_pred_all, average="macro")
train_rec = recall_score(train_true_all, train_pred_all, average="macro")
train_f1 = f1_score(train_true_all, train_pred_all, average="macro")
train_mcc = matthews_corrcoef(train_true_all, train_pred_all)
train_mae = mean_absolute_error(train_true_all, train_pred_all)
train_r2 = r2_score(train_true_all, train_pred_all)

print("\nTraining Metrics:")
print(f"Accuracy:  {train_acc:.4f}")
print(f"Precision: {train_prec:.4f}")
print(f"Recall:    {train_rec:.4f}")
print(f"F1 Score:  {train_f1:.4f}")
print(f"MCC:       {train_mcc:.4f}")
print(f"MAE:       {train_mae:.4f}")
print(f"R2 Score:  {train_r2:.4f}")

# Evaluation on Test
model.eval()
y_true, y_pred = [], []
y_prob = []  # For AUC

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["label"].to(DEVICE)

        outputs = model(input_ids, attention_mask)
        probs = torch.softmax(outputs, dim=1)
        preds = outputs.argmax(dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

test_acc = accuracy_score(y_true, y_pred)
test_prec = precision_score(y_true, y_pred, average="macro")
test_rec = recall_score(y_true, y_pred, average="macro")
test_f1 = f1_score(y_true, y_pred, average="macro")
test_mcc = matthews_corrcoef(y_true, y_pred)
test_mae = mean_absolute_error(y_true, y_pred)
test_r2 = r2_score(y_true, y_pred)

print("\nTest Metrics:")
print(f"Accuracy:  {test_acc:.4f}")
print(f"Precision: {test_prec:.4f}")
print(f"Recall:    {test_rec:.4f}")
print(f"F1 Score:  {test_f1:.4f}")
print(f"MCC:       {test_mcc:.4f}")
print(f"MAE:       {test_mae:.4f}")
print(f"R2 Score:  {test_r2:.4f}")

# Confusion Matrix Plot
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
classes = list(set(labels))
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

# ROC AUC Curve for Multiclass
y_true_bin = label_binarize(y_true, classes=classes)
y_prob = np.array(y_prob)
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_auc_score([y_true_bin[:, i]], [y_prob[:, i]])
    roc_auc[i] = roc_auc_score(y_true_bin[:, i], y_prob[:, i])

plt.figure()
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'brown'])
for i, color in zip(range(num_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color,
             label=f'Class {classes[i]} ROC curve (area = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - Multi-class')
plt.legend(loc="lower right")
plt.show()
